# Clear dimensions to be exposed through BI Reporting tool
1. Load and Clean Data

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import re

dates_df = pd.read_csv('../data/de_dates.csv')
transactions_df = pd.read_csv('../data/de_shop_transactions_20230821.csv')

# Replace special error values with None
transactions_df.replace('#ERROR!', None, inplace=True)

# Standardize phone numbers
def clean_phone_number(phone):
    if pd.isna(phone) or phone is None:
        return None
    phone = re.sub(r'\D', '', phone)  
    if len(phone) == 10:  
        return f'({phone[:3]}) {phone[3:6]}-{phone[6:]}'
    elif len(phone) == 11 and phone.startswith('1'):  
        return f'+1 ({phone[1:4]}) {phone[4:7]}-{phone[7:]}'
    else:
        return None
transactions_df['PHONE'] = transactions_df['PHONE'].apply(clean_phone_number)

# Validate numeric fields
transactions_df = transactions_df[pd.to_numeric(transactions_df['TOTAL_AMOUNT'], errors='coerce').notnull()]
transactions_df = transactions_df[pd.to_numeric(transactions_df['QUANTITYORDERED'], errors='coerce').notnull()]
transactions_df = transactions_df[transactions_df['QUANTITYORDERED'] != 0]

# Convert ORDERDATE to uniform date format (YYYY-MM-DD)
transactions_df['ORDERDATE'] = pd.to_datetime(transactions_df['ORDERDATE'], format='%m/%d/%Y %H:%M').dt.date
dates_df['CALENDAR_DATE'] = pd.to_datetime(dates_df['CALENDAR_DATE'], format='%Y-%m-%d').dt.date

2. Organize data into dimensions and fact table

In [2]:
# SQLite database in memory
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Load the cleaned data into SQLite tables
dates_df.to_sql('dates', conn, index=False, if_exists='replace')
transactions_df.to_sql('transactions', conn, index=False, if_exists='replace')

# Create dimensions and fact table
queries = {
    "create_customers_dimension": """
        CREATE TABLE customers_dim AS
        SELECT DISTINCT CUSTOMERNAME AS customer_name, PHONE AS phone, ADDRESSLINE1 AS address1, ADDRESSLINE2 AS address2,
                        CITY AS city, STATE AS state, POSTALCODE AS postal_code, COUNTRY AS country,
                        CONTACTLASTNAME AS contact_last_name, CONTACTFIRSTNAME AS contact_first_name, DEALSIZE AS deal_size
        FROM transactions;
    """,
    "create_products_dimension": """
        CREATE TABLE products_dim AS
        SELECT DISTINCT PRODUCTCODE AS product_code
        FROM transactions;
    """
}

for query_name, query in queries.items():
    cursor.execute(query)
    print(f"Executed {query_name}")

Executed create_customers_dimension
Executed create_products_dimension


In [3]:
create_transactions_fact_query = """
    CREATE TABLE transactions_fact AS
    SELECT t.ORDERNUMBER AS order_number, t.QUANTITYORDERED AS quantity_ordered, t.ORDERLINENUMBER AS order_line_number,
           t.TOTAL_AMOUNT AS total_amount, t.ORDERDATE AS order_date, t.QTR_ID AS quarter_id, t.MONTH_ID AS month_id,
           t.YEAR_ID AS year_id, t.PRODUCTCODE AS product_code, t.CUSTOMERNAME AS customer_name,
           d.FISCAL_YEAR AS fiscal_year, d.FISCAL_QUARTER AS fiscal_quarter, d.FISCAL_MONTH_NUMBER AS fiscal_month_number
    FROM transactions t
    LEFT JOIN dates d ON t.YEAR_ID = d.FISCAL_YEAR;
"""

cursor.execute(create_transactions_fact_query)

# Verify the result
transactions_fact_df = pd.read_sql_query("SELECT * FROM transactions_fact", conn)
print(transactions_fact_df[['order_date', 'fiscal_year', 'fiscal_quarter', 'fiscal_month_number']].head())


   order_date  fiscal_year  fiscal_quarter  fiscal_month_number
0  2003-02-24         2021               1                    1
1  2003-02-24         2021               1                    1
2  2003-02-24         2021               1                    1
3  2003-02-24         2021               1                    1
4  2003-02-24         2021               1                    1


3. Export Dimensions and Fact Tables to CSV Files so it can be used in the BI tool

In [4]:
# Export dimensions and fact table to CSV files
customers_dim_df = pd.read_sql_query("SELECT * FROM customers_dim", conn)
customers_dim_df.to_csv('../data/customers_dim.csv', index=False)

products_dim_df = pd.read_sql_query("SELECT * FROM products_dim", conn)
products_dim_df.to_csv('../data/products_dim.csv', index=False)

transactions_fact_df = pd.read_sql_query("SELECT * FROM transactions_fact", conn)
transactions_fact_df.to_csv('../data/transactions_fact.csv', index=False)

print("Dimensions and fact table exported to CSV files.")


Dimensions and fact table exported to CSV files.


4. Total Sales

In [5]:
total_sales_query = """
    SELECT SUM(total_amount) AS total_sales
    FROM transactions_fact
    WHERE total_amount > 0;
"""

total_sales_df = pd.read_sql_query(total_sales_query, conn)
print(total_sales_df)

   total_sales
0    509689054


5. Total Refunds

In [6]:
total_refunds_query = """
    SELECT SUM(total_amount) AS total_refunds
    FROM transactions_fact
    WHERE total_amount < 0;
"""

total_refunds_df = pd.read_sql_query(total_refunds_query, conn)
print(total_refunds_df)

   total_refunds
0     -523619180


6. Total Items 

In [7]:
total_items_query = """
    SELECT SUM(quantity_ordered) AS total_items
    FROM transactions_fact;
"""

total_items_df = pd.read_sql_query(total_items_query, conn)
print(total_items_df)


   total_items
0     36184162


7. Unit Price 

In [8]:
unit_price_query = """
    SELECT product_code, 
           SUM(ABS(total_amount)) OVER (PARTITION BY product_code) / 
           SUM(quantity_ordered) OVER (PARTITION BY product_code) AS unit_price
    FROM transactions_fact
    GROUP BY product_code;
"""

unit_price_df = pd.read_sql_query(unit_price_query, conn)
print(unit_price_df)

    product_code  unit_price
0       S10_1678          66
1       S10_1949          50
2       S10_2016          42
3       S10_4698          65
4       S10_4757           1
..           ...         ...
104    S700_3505          49
105    S700_3962          30
106    S700_4002          16
107     S72_1253          41
108     S72_3212          46

[109 rows x 2 columns]


8. Top 10 Customers 

In [9]:
top_customers_query = """
    SELECT t.customer_name, phone, address1, address2, city, state, postal_code, country, contact_last_name, contact_first_name, deal_size,
           SUM(total_amount) AS total_sales
    FROM transactions_fact AS t
    JOIN customers_dim AS c ON t.customer_name = c.customer_name
    WHERE total_amount > 0
    GROUP BY t.customer_name
    ORDER BY total_sales DESC
    LIMIT 10;
"""

top_customers_df = pd.read_sql_query(top_customers_query, conn)
print(top_customers_df)

                  customer_name           phone  \
0         Euro Shopping Channel            None   
1  Mini Gifts Distributors Ltd.  (415) 555-1450   
2     Souveniers And Things Co.            None   
3             Land of Toys Inc.  (212) 555-7818   
4       Dragon Souveniers, Ltd.            None   
5                AV Stores, Co.  (171) 555-1555   
6             La Rochelle Gifts            None   
7    Australian Collectors, Co.  (039) 520-4555   
8          Technics Stores Inc.  (650) 555-6809   
9      Danish Wholesale Imports            None   

                                  address1 address2        city     state  \
0                       C/ Moralzarzal, 86     None      Madrid      None   
1                          5677 Strong St.     None  San Rafael        CA   
2  Monitor Money Building, 815 Pacific Hwy  Level 6   Chatswood       NSW   
3                  897 Long Airport Avenue     None         NYC        NY   
4     Bronz Sok., Bronz Apt. 3/6 Tesvikiye     None  

9. Aggregate on Fiscal Dates 

In [10]:
fiscal_query = """
    SELECT fiscal_year, fiscal_quarter, fiscal_month_number,
           SUM(total_amount) AS total_sales,
           SUM(CASE WHEN total_amount < 0 THEN total_amount ELSE 0 END) AS total_refunds,
           SUM(quantity_ordered) AS total_items
    FROM transactions_fact
    GROUP BY fiscal_year, fiscal_quarter, fiscal_month_number;
"""

fiscal_aggregates_df = pd.read_sql_query(fiscal_query, conn)
print(fiscal_aggregates_df)

    fiscal_year  fiscal_quarter  fiscal_month_number  total_sales  \
0          2020               1                    1      -173964   
1          2020               1                    2      -217455   
2          2020               1                    3      -173964   
3          2020               2                    4      -173964   
4          2020               2                    5      -217455   
5          2020               2                    6      -173964   
6          2020               3                    7      -173964   
7          2020               3                    8      -217455   
8          2020               3                    9      -173964   
9          2020               4                   10      -173964   
10         2020               4                   11      -217455   
11         2020               4                   12      -173964   
12         2021               1                    1       720104   
13         2021               1   